In [1]:
import pandas as pd
import ztarget
import model_engine
import json

ztarget.__version__, model_engine.__version__

('1.2.1', 'v1.13.0')

In [2]:
# Zest analysis data 
ana = pd.read_parquet('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/processed/supporting_artifacts/analysis_data.parquet')
ana.shape

(651809, 179)

In [4]:
# PenFed Sent Data
df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')
df_raw.shape

/tmp/ipykernel_6255/4184390068.py:2: DtypeWarning: Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')


(651816, 126)

# Funded Population

Joint Application: \
PenFed Funded Trade: Deduplicate to have one record.\
Proxy Trade: Find the proxy for primary and coborrower, then choose trade opened closest to app received date.\
Score/Attribute: Deduplicate by using the applicant with higher FICO/Vantage score. Use primary if they have the same score.\
Expected Result: One unique trade/score/attribute for joint application\
Total Count: 651,816 before joint application deduplication. \
Total Counts after joint application deduplication: 579,000.


In [5]:
# Drop Duplicates 

df_raw = df_raw.sort_values(['Credit_Risk_Score', 'Role_Code'], ascending=[False, True])
df = df_raw.drop_duplicates(['Application_Number'])
df.shape

(579000, 126)

Filter: \
First deduplicate for Joint applications with logic above.\
Exclude Application_Status  in ‘Refer’, ‘Processing’, ‘Exception’.\
Exclude FICO < 610 (Using higher fico for joint application)


In [6]:
# Majority of Refer is duplicate application
df = df[df['Application_Status'].isin(['Refer', 'Processing', 'Exception'])==False]
print(df.shape) # 561,137
df = df[df['Max_Credit_Risk_Score']>=610] # 537656
print(df.shape) # 537, 652


(561137, 126)
(537652, 126)


Additional Filters on PenFed Funded Loan (when PenFed_Funded=1)
Exclude Insured Loan (exclude accounts with a lend pro number) 
Exclude Canceled and Check Not Cashed 
Exclude 3rd Party Fraud
PenFed Funded Only After filters in the filter section. 


In [7]:
ana['benchmark_scores_from_client'] = pd.to_numeric(ana['benchmark_scores_from_client'], 
                                                    errors='coerce') 

ana['Max_Credit_Risk_Score'] = pd.to_numeric(ana['Max_Credit_Risk_Score'], 
                                             errors='coerce')  

ana[['benchmark_scores_from_client', 'Max_Credit_Risk_Score']].describe()

,benchmark_scores_from_client,Max_Credit_Risk_Score
count,635120.000000,637646.000000
mean,724.773265,731.987385
std,70.047235,65.207109
min,339.000000,339.000000
25%,674.000000,682.000000
50%,729.000000,735.000000
75%,781.000000,785.000000
max,850.000000,850.000000


# Unfunded Proxy Target

In [8]:
from ztarget import EQTargetGenerator, TUTargetGenerator, EXPTargetGenerator
from ztarget.target_generator import final_target
from model_engine.assets.utils import load_asset
from itertools import product

pd.options.display.max_columns = None
pd.options.display.float_format = '{:.4f}'.format

/home/lyt/.conda/envs/penfed_psp/lib/python3.10/site-packages/zamlexplain/utils_compile.py:23: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC
/home/lyt/.conda/envs/penfed_psp/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/lyt/.conda/envs/penfed_psp/lib/python3.10/site-packages

In [9]:
ana.flgFunded.value_counts()

flgFunded
0    497471
1    154338
Name: count, dtype: int64

In [10]:
df_raw['PenFed_Funded'].value_counts()

PenFed_Funded
0    497478
1    154338
Name: count, dtype: int64

In [11]:
# Zest Analysis Application Numbers , PenFed Unfunded Application Numbers 

ana[ana.flgFunded == 0]['appId'].nunique(), df_raw[df_raw.PenFed_Funded == 0]['Application_Number'].nunique()

(445111, 445117)

In [12]:
path = 's3://power-client-data-staging/CLIENT/PARSED/DATA/BUREAU%3Dequifax/FORMAT%3Dcms_6/TABLE%3Dtrade/PULL_NAME%3D20250529_penfed/ARCHIVE_DATE%3D2025-04-30/'.replace('%3D', '=')
perf_trade = pd.read_parquet(path)

Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.



In [13]:
unfunded_ana = ana[ana.flgFunded == 0]
unfunded_ana.shape

(497471, 179)

In [14]:
unfunded_trade = perf_trade[perf_trade.ZEST_KEY.isin(unfunded_ana.index)]
unfunded_trade.ZEST_KEY.nunique()

489882

## Auto Part 1 – Auto Loan Proxy

Return a record for all Auto Loans (exclude leases) opened within +/-3 months from app month

Fields returned for each proxy trade:
- MOB6 Ever 60+DPD/BNK/Repossession/Charged-off
- MOB12 Ever 90+DPD/BNK/Repossession/Charged-off (General Bad PenFed used for model development)
- MOB12 Ever 120+DPD/Repossession/Charged-off (Simulate PenFed current Charge off Policy. Does not include BNK)
- MOB18 Ever 90+DPD/BNK/Repossession/Charged-off
- MOB18 Ever 120+DPD/Repossession/Charged-off
- MOB24 Ever 120+DPD/Repossession/Charged-off
- Payment history grid (pulled from current archive)
	
If there are multiple proxy trades for an individual application, we want to keep all of them as reference.


In [15]:
dqs = ["DQ30", "DQ60", "DQ90", "DQ120", "CO"]
time_window = [3, 6, 9, 12, 15, 18, 24, 30, 36]
default_proxy_time_window = [-30, 90]
eq_trade_json = load_asset('equifax/cms_6/trade.json')


# exclude 3A: Auto Lease 
# exclude 11: recreational merchandise

proxy_account_type = ['00']
target_generator = EQTargetGenerator(join_key='ZEST_KEY', app_id='appId', app_date='appDate', 
                                      proxy_time_window = default_proxy_time_window, 
                                      proxy_account_type = proxy_account_type,
                                      **eq_trade_json['info'],
                                     proxy = True)
target_generator.proxy_account_type

['00']

In [16]:
cols = ['appId', 'appDate', 'flgFunded', 'flgApproved',]
unfunded_trade = unfunded_trade.merge(unfunded_ana[cols], left_on = 'ZEST_KEY', 
                       right_index = True)

In [17]:
proxy_date_df = target_generator.get_dqDates(unfunded_trade)

--- Transforming date formats ---
Warning! Cannot covert CLOSED_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_BEFORE_HISTORY to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert LAST_PAYMENT_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DFD_DLA to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_3 to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DMD_REPORTED to datetime using %m%d%Y format! Please check the asset dates format!
T

In [50]:
unfunded_ana.shape

(497471, 179)

In [18]:
proxy_date_df.ZEST_KEY.nunique() / unfunded_ana.shape[0]

0.29134361600977743

In [19]:
proxy_date_df.ZEST_KEY.nunique(), proxy_date_df.shape

(144935, (166029, 105))

In [30]:
proxy_date_df.to_parquet('/d/shared/users/lyt/penfed_poc1_unfunded_proxy_targets/full_auto_loan_proxy.parquet')

In [31]:
proxy_date_df = pd.read_parquet('/d/shared/users/lyt/penfed_poc1_unfunded_proxy_targets/full_auto_loan_proxy.parquet')
proxy_date_df.shape

(166029, 105)

In [22]:
with open(f"/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/bureau_config.json", "r") as file: 
    config = json.load(file)
config

{'conf': {'BUREAU': 'equifax',
  'FORMAT': 'cms_6',
  'PULL_DATE': '2025-05-29',
  'PULL_NAME': '20250529_penfed',
  'PERFORMANCE_DATE': '2025-04-30'}}

In [23]:
pull_date = config['conf']['PULL_DATE']
proxy_target = target_generator.generate_target(proxy_date_df, 
                                         target=dqs, 
                                         time_window=time_window, 
                                         query_date= pull_date)


DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

In [24]:
proxy_target.shape

(166029, 219)

In [46]:
penfed_required_col_list = ['ZEST_KEY', 'appId', 'appDate', 'flgFunded',
                             'flgApproved','DATE_OPENED', 
                             'proxy_DQ60_m6','proxy_DQ60_m6_status', 
                             'proxy_DQ90_m12','proxy_DQ90_m12_status',
                             'proxy_DQ120_m12','proxy_DQ120_m12_status',
                             'proxy_DQ90_m18','proxy_DQ90_m18_status',
                             'proxy_DQ120_m18','proxy_DQ120_m18_status',
                             'proxy_DQ120_m24','proxy_DQ120_m24_status',
                             'PAYMENT_HISTORY_1_24', 'PAYMENT_HISTORY_25_36','PAYMENT_HISTORY_37_48']

In [47]:
unfunded_ana.shape

(497471, 179)

In [48]:
cols = ['ZEST_KEY', 'appId', 'appDate', 'flgFunded', 'flgApproved',]

proxy_loan_required_by_penfed = proxy_target.merge(unfunded_ana.reset_index()[cols], 
                                                   how = 'left')[penfed_required_col_list]
proxy_loan_required_by_penfed.shape

(166029, 21)

In [49]:
proxy_loan_required_by_penfed.to_parquet('/d/shared/users/lyt/penfed_poc1_unfunded_proxy_targets/proxy_loan_required_by_penfed.parquet')

In [32]:
proxy_target.to_parquet('/d/shared/users/lyt/penfed_poc1_unfunded_proxy_targets/auto_proxy_targets.parquet')

In [34]:
# Merging with unfunded analysis data 

final_proxy_target = unfunded_ana.reset_index()[cols].merge(proxy_target, how = 'left')
final_proxy_target.shape

(518565, 219)

In [35]:
final_proxy_target.ZEST_KEY.nunique()

497471

In [36]:
final_proxy_target.to_parquet('/d/shared/users/lyt/penfed_poc1_unfunded_proxy_targets/final_auto_proxy_targets.parquet')

## Auto Part 2 – Waterfall Proxy – Only Proceed to this step if no data is found in Part 1 

Return one record max per applicant (the first that is found in the waterfall)

1. Non-Mortgage Secured Loan opened -1 to 1 month from application month
2. Non-Mortgage Secured Loan opened -1 to 3 month from application month
3. Non-Mortgage Secured Loan opened -3 to 3 month from application month 
4. Existing Auto Loan (must be paid as agreed at app month)
5. Existing Personal Loan (must be paid as agreed at app month)
6. Any Open Loan (must be paid as agreed at app month)

Duplicate treatment logic (for Part 2 only)
Choose worst performing trade (60+DPD M12), next choose trade opened closest to app month 
Else random select from duplicate trades

Fields returned for the single proxy, if found:
Waterfall # that generated the trade 
- Date Opened, Loan Amount, Terms, Estimated APR
- MOB6 Ever 60+DPD/BNK/Repossession/Charged-off
- MOB12 Ever 90+DPD/BNK/Repossession/Charged-off (General Bad PenFed used for model development)
- MOB12 Ever 120+DPD/Repossession/Charged-off (Simulate PenFed current Charge off Policy. Does not include BNK)
- MOB18 Ever 90+DPD/BNK/Repossession/Charged-off
- MOB18 Ever 120+DPD/Repossession/Charged-off
- MOB24 Ever 120+DPD/Repossession/Charged-off
- Payment history grid (pulled from current archive)


### A. Non-Mortgage Secured Loans

In [27]:
unfunded_ana_v1 = unfunded_ana[~unfunded_ana.index.isin(proxy_date_df.ZEST_KEY)]

In [25]:
# 02: secured
# 22: Secured By Household Goods
# 23: Secured By Household Goods/Collateral

default_proxy_time_window = [-90, 90]
proxy_account_type = ['02', '22', '23']
target_generator = EQTargetGenerator(join_key='ZEST_KEY', app_id='appId', app_date='appDate', 
                                      proxy_time_window = default_proxy_time_window, 
                                      proxy_account_type = proxy_account_type,
                                      **eq_trade_json['info'],
                                     proxy = True)
target_generator.proxy_account_type

['02', '22', '23']

In [28]:
unfunded_trade_v1 = unfunded_trade[unfunded_trade.ZEST_KEY.isin(unfunded_ana_v1.index)]

proxy_date_df_v1 = target_generator.get_dqDates(unfunded_trade_v1)

--- Transforming date formats ---
Warning! Cannot covert CLOSED_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_BEFORE_HISTORY to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert LAST_PAYMENT_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DFD_DLA to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_3 to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DMD_REPORTED to datetime using %m%d%Y format! Please check the asset dates format!
T

In [30]:
# Exclude Mortgage Loans 
print(proxy_date_df_v1.ZEST_KEY.nunique())
proxy_date_df_v1.PORTFOLIO_TYPE.value_counts()

8035


PORTFOLIO_TYPE
I    9044
*     132
O      38
M      11
Name: count, dtype: int64

In [31]:
# Exclude Mortgage Loans 
proxy_date_df_v1 = proxy_date_df_v1[proxy_date_df_v1.PORTFOLIO_TYPE != 'M']
proxy_date_df_v1.ZEST_KEY.nunique()

8024

In [35]:
proxy_date_df_v1['days_diff'] = (proxy_date_df_v1['DATE_OPENED'] - proxy_date_df_v1['appDate']).dt.days

def classify_loan_timing(days_diff):
    if -30 <= days_diff <= 30:
        return 1
    elif -30 <= days_diff <= 90:
        return 2
    elif -90 <= days_diff <= 90:
        return 3
    else:
        return np.nan  # or 0 or any other indicator for not falling into these windows

proxy_date_df_v1['Waterfall_Number'] = proxy_date_df_v1['days_diff'].apply(classify_loan_timing)

In [36]:
proxy_date_df_v1['Waterfall_Number'].value_counts(dropna = False)

Waterfall_Number
1    3703
2    2867
3    2769
Name: count, dtype: int64

In [38]:
min_waterfall = proxy_date_df_v1.groupby('ZEST_KEY')['Waterfall_Number'].transform('min')

# Keep only rows where the waterfall_number is the minimum for that ZEST_KEY
proxy_date_df_v1 = proxy_date_df_v1[proxy_date_df_v1['Waterfall_Number'] == min_waterfall].copy()

In [43]:
proxy_date_df_v1.ZEST_KEY.nunique()

8024

In [79]:
# proxy_date_df_v1.to_parquet('./proxy_trades_part2_1.parquet')

### B.Existing Loans

In [49]:
unfunded_ana_v2 = unfunded_ana_v1[~unfunded_ana_v1.index.isin(proxy_date_df_v1.ZEST_KEY)]
unfunded_trade_v2 = unfunded_trade[unfunded_trade.ZEST_KEY.isin(unfunded_ana_v2.index)]

In [50]:
unfunded_trade_v2.ZEST_KEY.nunique()

305273

In [124]:
import boto3 
bucket_name = 'power-client-data-staging'
s3 = boto3.client("s3")

pre = f"PREPROCESSED/DATA/BUREAU%3Dequifax/FORMAT%3Dcms_6/TABLE%3Dtrade/VERSION%3Dv2/CLIENT%3Dpenfed/PRODUCT%3Dautoindirect/PULL_DATE%3D2025-05-29/PULL_NAME%3D20250529_penfed/ME_VERSION%3Dv1.13.0/".replace("%3D", "=")
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=pre, Delimiter="/")

In [125]:
trade = pd.DataFrame()

for c in response['CommonPrefixes']:
    cur_table_path = "s3://power-client-data-staging/"+c['Prefix']
    print(cur_table_path)
    temp = pd.read_parquet(cur_table_path)
    trade = pd.concat([trade, temp])

s3://power-client-data-staging/PREPROCESSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=trade/VERSION=v2/CLIENT=penfed/PRODUCT=autoindirect/PULL_DATE=2025-05-29/PULL_NAME=20250529_penfed/ME_VERSION=v1.13.0/ARCHIVE_DATE=2020-12-31/
s3://power-client-data-staging/PREPROCESSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=trade/VERSION=v2/CLIENT=penfed/PRODUCT=autoindirect/PULL_DATE=2025-05-29/PULL_NAME=20250529_penfed/ME_VERSION=v1.13.0/ARCHIVE_DATE=2021-03-31/
s3://power-client-data-staging/PREPROCESSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=trade/VERSION=v2/CLIENT=penfed/PRODUCT=autoindirect/PULL_DATE=2025-05-29/PULL_NAME=20250529_penfed/ME_VERSION=v1.13.0/ARCHIVE_DATE=2021-06-30/
s3://power-client-data-staging/PREPROCESSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=trade/VERSION=v2/CLIENT=penfed/PRODUCT=autoindirect/PULL_DATE=2025-05-29/PULL_NAME=20250529_penfed/ME_VERSION=v1.13.0/ARCHIVE_DATE=2021-09-30/
s3://power-client-data-staging/PREPROCESSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=trade/VERSION

In [149]:
proxy_date_df['LAST_PAYMENT_DATE']

119        2025-03-01
176        2025-03-01
198        2023-05-01
199        2023-11-01
365        2025-03-01
              ...    
19335731   2025-03-01
19335732   2025-03-01
19335896   2025-02-01
19336092   2025-03-01
19336098   2024-11-01
Name: LAST_PAYMENT_DATE, Length: 211841, dtype: datetime64[ns]

In [140]:
trade = trade[trade.ZEST_KEY.isin(unfunded_ana_v2.index)]
trade.ZEST_KEY.nunique(), unfunded_ana_v2.shape

(304181, (312862, 179))

In [199]:
trade.ACTIVITY_DESIGNATOR.value_counts()

ACTIVITY_DESIGNATOR
B    2729596
T     248577
C     100589
P      24336
D      20489
L       9593
R       2859
Name: count, dtype: int64

In [142]:
# Filter for currently paid as agreed accounts 

trade_filtered = trade[trade['RATE_STATUS_CODE'] == '1']
trade_filtered.ZEST_KEY.nunique()

304044

In [170]:
trade_filtered['ACCOUNT_TYPE'].value_counts().head(10)

ACCOUNT_TYPE
18    1480008
07    1311093
00     892350
12     618541
0G     306975
01     251508
26     229563
02     110943
3A      79783
19      70276
Name: count, dtype: int64

In [161]:
import numpy as np
def udpate_date_formats(data, dates): 
    for date_col, date_format in dates.items():
        if np.issubdtype(data[date_col].dtype, np.datetime64):
            print(f'{date_col} is already of datetime format! Will skip the converting process')
            continue
        data[date_col] = data[date_col].astype('str')
        for date_col, date_format in dates.items(): 
            if date_format == '%m%d%Y':
                idx = data[date_col].str[2:4] == '00'
                data.loc[idx, date_col] = data.loc[idx, date_col].str[0:2] + '01' + data.loc[idx, date_col].str[4:]
                data[date_col] = data[date_col].replace('        ', np.nan)
            try:
                data[date_col] = pd.to_datetime(data[date_col], format=date_format, errors='raise')
            except:
                print(f'Warning! Cannot covert {date_col} to datetime using {date_format} format! Please check the asset dates format!' )
                print('Trying to convert datetime using inferred datetime format...')
                data[date_col] = pd.to_datetime(data[date_col],format=date_format, errors='coerce')
    return data         

In [162]:
dates = {
    'DATE_REPORTED': '%m%d%Y',
    'LAST_PAYMENT_DATE': '%m%d%Y'
}

trade_filtered = udpate_date_formats(trade_filtered, dates)

DATE_REPORTED is already of datetime format! Will skip the converting process
LAST_PAYMENT_DATE is already of datetime format! Will skip the converting process


In [164]:
trade_filtered.ZEST_KEY.nunique()

304044

In [446]:
def get_time_diff(reference_date_series, date_series, units):
    if units == "M":
        # According pandas==2.0.2, 1 month is exactly 30 days. The number
        # '30.436875' was reversed engineering from pandas 1.X. This number
        # is about 365.2422 (days) / 12 (months).
        return (reference_date_series - date_series) / pd.Timedelta(30.436875, "D")
    else:
        return (reference_date_series - date_series) / np.timedelta64(1, units)


In [222]:
# Month difference between appDate and date of request

trade_filtered['request_date_diff'] =  get_time_diff(trade_filtered['appDate'], trade_filtered['DATE_OF_REQUEST'], units = 'M')
trade_filtered['request_date_diff'].describe()

count   5653828.0000
mean          1.6145
std           0.8688
min           0.0329
25%           0.8871
50%           1.6099
75%           2.3656
max           3.2198
Name: request_date_diff, dtype: float64

In [229]:
trade_filtered['DATE_OF_REQUEST'].value_counts()

DATE_OF_REQUEST
2022-03-29    567143
2021-12-28    493476
2024-03-26    460577
2023-12-26    427018
2021-09-28    389096
2023-03-28    386543
2023-06-27    373007
2024-06-25    341287
2023-09-26    333301
2022-12-27    316118
2022-06-28    308148
2022-09-27    275987
2021-06-29    260070
2024-12-31    244020
2021-03-30    197917
2020-12-29    159522
2024-09-24    120598
Name: count, dtype: int64

In [223]:
# Month difference between payment date and date of request 

trade_filtered['payment_date_diff'] =  get_time_diff(trade_filtered['DATE_OF_REQUEST'], trade_filtered['LAST_PAYMENT_DATE'], units = 'M')
trade_filtered['payment_date_diff'].describe()

count   5098919.0000
mean         39.9468
std          48.1579
min           0.7557
25%           1.8727
50%          17.8073
75%          68.7981
max         471.7961
Name: payment_date_diff, dtype: float64

In [230]:
# Account not closed indicated by CLOSED_DATE

trade_filtered[trade_filtered.CLOSED_DATE.notna()].payment_date_diff.describe()

count   2619908.0000
mean         69.8833
std          47.7217
min           0.7557
25%          32.8220
50%          62.8514
75%          98.8275
max         471.7961
Name: payment_date_diff, dtype: float64

In [225]:
# Account is closed indicated by CLOSED_DATE

trade_filtered[trade_filtered.CLOSED_DATE.isnull()].payment_date_diff.describe()

count   2479011.0000
mean          8.3088
std          20.3789
min           0.7557
25%           1.7742
50%           1.8727
75%           3.8769
max         389.8889
Name: payment_date_diff, dtype: float64

In [210]:
# Account is closed indicated by CLOSED_DATE

trade_filtered[(~trade_filtered.CLOSED_DATE.isnull())][['report_date_diff', 'payment_date_diff']].describe()

,report_date_diff,payment_date_diff
count,2935370.0000,2619908.0000
mean,52.4012,71.5052
std,33.0762,47.7345
min,0.1314,0.9856
25%,23.9841,34.2019
50%,49.7094,64.4941
75%,78.5560,100.3388
max,121.9573,474.3588


In [211]:
# Account not closed indicated by CLOSED_DATE

trade_filtered[trade_filtered.CLOSED_DATE.isnull()][['report_date_diff', 'payment_date_diff']].describe()

,report_date_diff,payment_date_diff
count,2718458.0000,2479011.0000
mean,5.4583,9.9135
std,16.0214,20.4046
min,0.0986,0.9856
25%,1.6099,2.7270
50%,2.4313,3.7455
75%,3.2198,5.6839
max,121.9573,392.0573


In [236]:
# Filter for currently active accounts - payment and application date difference is less than 4 months and no close_date 

active_trade = trade_filtered[ (trade_filtered['payment_date_diff'] <=1)]
active_trade.ZEST_KEY.nunique()

238223

In [237]:
active_trade['ACCOUNT_TYPE'].value_counts().head(10)

ACCOUNT_TYPE
18    300396
07    109124
0G     90441
00     45179
26     28391
12      6780
19      5375
01      4595
2A      3637
25      3108
Name: count, dtype: int64

4. Existing Auto Loan (must be paid as agreed at app month)
5. Existing Personal Loan (must be paid as agreed at app month)
6. Any Open Loan (must be paid as agreed at app month)

In [238]:
pl_types =['01',
    '02',
    '03',
    '06',
    '0A',
    '0F',
    '11',
    '13',
    '15',
    '20',
    '29',
    '47',
    '4D',
    '70',
    '7B',
    '92',
    '78',
    '91']
auto_types = ['3A', '00', '11']

In [239]:
def classify_loan_type(account_col):
    if account_col in auto_types:
        return 4
    elif account_col in pl_types:
        return 5
    else:
        return 6  # or 0 or any other indicator for not falling into these windows

active_trade['Waterfall_Number'] = active_trade['ACCOUNT_TYPE'].apply(classify_loan_type)

In [240]:
active_trade['Waterfall_Number'].value_counts()

Waterfall_Number
6    552105
4     47359
5      7630
Name: count, dtype: int64

In [244]:
active_trade.ZEST_KEY.nunique()

238223

In [245]:
min_waterfall = active_trade.groupby('ZEST_KEY')['Waterfall_Number'].transform('min')

# Keep only rows where the waterfall_number is the minimum for that ZEST_KEY
existing_trade = active_trade[active_trade['Waterfall_Number'] == min_waterfall].copy()
existing_trade.ZEST_KEY.nunique()

238223

In [246]:
existing_trade.shape

(502291, 69)

In [295]:
existing_trade.ZEST_KEY.value_counts().head(20)

ZEST_KEY
86707439_2_229_22     35
77596429_1_229_22     34
86707439_1_229_22     33
82239072_2_229_22     33
95294297_1_229_22     30
84322710_1_229_22     30
73992434_1_229_22     30
70472514_1_229_22     28
74166730_1_229_22     28
75748977_1_229_22     27
109622340_1_229_22    27
81680965_1_229_22     27
83037846_1_229_22     26
83130685_1_229_22     26
79168354_1_229_22     26
81715181_1_229_22     26
106317965_2_229_22    26
72052895_2_229_22     25
76751342_1_229_22     25
84267270_1_229_22     25
Name: count, dtype: int64

In [410]:
# # Experiment with including portfolio type 
# tempo = existing_trade[['ZEST_KEY', 'DATE_OPENED', 'ACCOUNT_TYPE', 'CUSTOMER_NUMBER', 'ECOA_DESIGNATOR', 'PORTFOLIO_TYPE', 'Waterfall_Number']].merge(unfunded_trade,
#      on = ['ZEST_KEY', 'DATE_OPENED', 'ACCOUNT_TYPE', 'CUSTOMER_NUMBER', 'ECOA_DESIGNATOR','PORTFOLIO_TYPE', ],
#      how = 'inner',
#      suffixes=('_app', ''))
# tempo.ZEST_KEY.nunique()/ existing_trade.ZEST_KEY.nunique()

0.9591811034199049

In [421]:
# merged_trade[(~merged_trade.ZEST_KEY.isin(tempo.ZEST_KEY)) & (merged_trade.PORTFOLIO_TYPE== 'I')]

,ZEST_KEY,DATE_OPENED,ACCOUNT_TYPE,CUSTOMER_NUMBER,ECOA_DESIGNATOR,Waterfall_Number,SEG_SEQ,SEG_PARENT,SEG_PARENT_SEQ,SEGMENT_TYPE,CUSTOMER_NAME,DATE_REPORTED,HIGH_CREDIT,CREDIT_LIMIT,BALANCE,PAST_DUE_AMOUNT,PORTFOLIO_TYPE,RATE_STATUS_CODE,AUTOMATED_UPDATE_INDICATOR,MONTHS_REVIEWED,ACCOUNT_NUMBER,30_DAY_COUNTERS,60_DAY_COUNTERS,90+_DAY_COUNTERS,PREVIOUS_HIGH_RATE_1,PREVIOUS_HIGH_DATE_1,PREVIOUS_HIGH_RATE_2,PREVIOUS_HIGH_DATE_2,PREVIOUS_HIGH_RATE_3,PREVIOUS_HIGH_DATE_3,DFD_DLA,NARRATIVE_CODE_1,NARRATIVE_CODE_2,NARRATIVE_CODE_3,NARRATIVE_CODE_4,LAST_PAYMENT_DATE,CLOSED_DATE,DMD_REPORTED,ACTUAL_PAYMENT_AMOUNT,SCHEDULED_PAYMENT_AMOUNT,TERMS_FREQUENCY,TERMS_DURATION,INDICATOR,NAME,CREDITOR_CLASSIFICATION,ACTIVITY_DESIGNATOR,ORIGINAL_CHARGE_OFF_AMOUNT,DEFERRED_PAYMENT_START_DATE,BALLOON_PAYMENT_AMOUNT,BALLOON_PAYMENT_DUE_DATE,MORTGAGE_ID_NUMBER,PAYMENT_HISTORY_1_24,PAYMENT_HISTORY_25_36,PAYMENT_HISTORY_37_48,PREVIOUS_HIGH_RATE_BEFORE_HISTORY,PREVIOUS_HIGH_DATE_BEFORE_HISTORY,NARRATIVE_CODE_1X,NARRATIVE_CODE_2X,NARRATIVE_CODE_3X,NARRATIVE_CODE_4X,RECENT_TRADE_FLAG,QUALIFYING_FLAG,ARCHIVE_DATA,appId,appDate,flgFunded,flgApproved
106732,78573212_1_229_22,08102021,18,FC,I,6,4,FULL-Header,4,PT,None,03312025,None,None,000010929,000010929,I,9,*,43,None,01,01,17,9,022025,9,012025,9,122024,08002023,DB,CW,None,None,06002023,None,01002024,None,None,None,0001,None,None,None,C,000007892,None,None,None,None,999999999999/995543211111,/111111111111,/11111E******,*,None,067,065,None,None,Y,Y,None,78573212,2022-01-28,0,1


In [429]:
merged_trade = existing_trade[['ZEST_KEY', 'DATE_OPENED', 'ACCOUNT_TYPE', 'CUSTOMER_NUMBER', 'ECOA_DESIGNATOR', 'PORTFOLIO_TYPE', 'Waterfall_Number']].merge(unfunded_trade,
     on = ['ZEST_KEY', 'DATE_OPENED', 'ACCOUNT_TYPE', 'CUSTOMER_NUMBER', 'ECOA_DESIGNATOR','PORTFOLIO_TYPE', ],
     how = 'inner',
     suffixes=('_app', ''))

In [430]:
merged_trade.ZEST_KEY.nunique() / existing_trade.ZEST_KEY.nunique()

0.9591811034199049

In [433]:
# temp1 = existing_trade[
# existing_trade.ZEST_KEY == '78573212_1_229_22'
# ][['ZEST_KEY', 'DATE_OPENED', 'HIGH_CREDIT', 'ACCOUNT_TYPE', 'CUSTOMER_NUMBER', 'PORTFOLIO_TYPE', 'ECOA_DESIGNATOR',
#   'CREDIT_LIMIT']]
# temp1

In [434]:
# temp2 = perf_trade[
# perf_trade.ZEST_KEY == '78573212_1_229_22'
# ][['ZEST_KEY', 'DATE_OPENED', 'HIGH_CREDIT', 'ACCOUNT_TYPE', 'CUSTOMER_NUMBER', 'PORTFOLIO_TYPE', 'ECOA_DESIGNATOR',
#   'CREDIT_LIMIT']]
# temp2

In [435]:
# temp_m = temp1.merge(temp2,
#                      on = ['ZEST_KEY', 'ACCOUNT_TYPE','DATE_OPENED','CUSTOMER_NUMBER', 'ECOA_DESIGNATOR'],
#                      how = 'inner',
#                      suffixes=('_app', '_perf'))
# temp_m[['ZEST_KEY','ACCOUNT_TYPE','DATE_OPENED',
#  'CREDIT_LIMIT_app',
#  'CREDIT_LIMIT_perf',
#  'CUSTOMER_NUMBER',
#  'ECOA_DESIGNATOR',
#  'HIGH_CREDIT_app',
#  'HIGH_CREDIT_perf',
#  'PORTFOLIO_TYPE_app',
#  'PORTFOLIO_TYPE_perf']]

In [436]:
all_types = merged_trade.ACCOUNT_TYPE.unique().tolist()
all_types

['01',
 '0G',
 '26',
 '18',
 '25',
 '07',
 '8A',
 '12',
 '00',
 '5B',
 '2A',
 '2C',
 '19',
 '02',
 '90',
 '3A',
 '06',
 '15',
 '29',
 '20',
 '17',
 '89',
 '11',
 '47',
 '5A',
 '91',
 '50',
 None,
 '13',
 '04',
 '08',
 '6D',
 '93',
 '03',
 '95',
 '10',
 '6B',
 '8B',
 '70',
 '0F',
 '7B',
 '0A',
 '9A']

In [437]:
default_proxy_time_window = [-36500, 36500]
proxy_account_type = all_types
target_generator = EQTargetGenerator(join_key='ZEST_KEY', app_id='appId', app_date='appDate', 
                                      proxy_time_window = default_proxy_time_window, 
                                      proxy_account_type = proxy_account_type,
                                      **eq_trade_json['info'],
                                     proxy = True)
target_generator.proxy_account_type

['01',
 '0G',
 '26',
 '18',
 '25',
 '07',
 '8A',
 '12',
 '00',
 '5B',
 '2A',
 '2C',
 '19',
 '02',
 '90',
 '3A',
 '06',
 '15',
 '29',
 '20',
 '17',
 '89',
 '11',
 '47',
 '5A',
 '91',
 '50',
 None,
 '13',
 '04',
 '08',
 '6D',
 '93',
 '03',
 '95',
 '10',
 '6B',
 '8B',
 '70',
 '0F',
 '7B',
 '0A',
 '9A']

In [438]:
existing_trade = existing_trade.reset_index(drop = True)

In [439]:
proxy_date_df_v2 = target_generator.get_dqDates(existing_trade)

--- Transforming date formats ---
Warning! Cannot covert CLOSED_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_BEFORE_HISTORY to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
LAST_PAYMENT_DATE is already of datetime format! Will skip the converting process
Warning! Cannot covert DFD_DLA to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_3 to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DMD_REPORTED to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DEFERRE

In [440]:
proxy_date_df_v2.ZEST_KEY.nunique(), existing_trade.ZEST_KEY.nunique()

(238222, 238223)

In [447]:
proxy_date_df_v2['appDate_diff'] = get_time_diff(proxy_date_df_v2['appDate'], proxy_date_df_v2['DATE_OPENED'], units = 'D')
proxy_date_df_v2['appDate_diff'].describe()

count   502289.0000
mean      1677.1322
std       1933.5958
min         11.0000
25%        429.0000
50%       1004.0000
75%       2174.0000
max      22906.0000
Name: appDate_diff, dtype: float64

In [470]:
proxy_date_df_v2[proxy_date_df_v2.DateDQ60.notna()].ZEST_KEY.value_counts()

ZEST_KEY
70584015_1_229_22     10
76160206_1_229_22      9
85997553_1_229_22      9
80644198_1_229_22      8
77163344_1_229_22      7
                      ..
76856546_1_229_22      1
77378160_1_229_22      1
77378360_1_229_22      1
77453495_1_229_22      1
109785794_2_229_22     1
Name: count, Length: 5660, dtype: int64

In [464]:
proxy_date_df_v2_wo_DQ60 = proxy_date_df_v2[proxy_date_df_v2.DateDQ60.isnull()]
proxy_date_df_v2_wo_DQ60.ZEST_KEY.nunique()

236668

In [465]:
proxy_date_df_v2_wo_DQ60 = proxy_date_df_v2_wo_DQ60.sort_values(['ZEST_KEY', 'appDate_diff'], ascending = [True, True]).drop_duplicates(['ZEST_KEY'],keep='first')
proxy_date_df_v2_wo_DQ60.appDate.min(), proxy_date_df_v2_wo_DQ60.appDate.max()

(Timestamp('2021-01-01 00:00:00'), Timestamp('2025-03-31 00:00:00'))

In [466]:
proxy_date_df_v2_wo_DQ60.appDate_diff.describe()

count   236668.0000
mean      1011.6747
std       1392.3046
min         11.0000
25%        257.0000
50%        536.0000
75%       1163.0000
max      21084.0000
Name: appDate_diff, dtype: float64